In [0]:
from google.colab import files
import io

import pandas as pd
from scipy.spatial.distance import cosine
import numpy

In [2]:
from google.colab import files
uploaded = files.upload()

Saving u.item to u (2).item
Saving u.user to u (2).user
Saving u.data to u (3).data


In [0]:
dataCols = ['User', 'Item', 'Rating', 'Timestamp']
itemCols = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
            'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 
            'Sci-Fi', 'Thriller', 'War', 'Western']
userCols = ['user id', 'age', 'gender', 'occupation', 'zip code']

genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


data = pd.read_csv(io.BytesIO(uploaded['u.data']), '\t', header=None, names=dataCols)
item = pd.read_csv(io.BytesIO(uploaded['u.item']), '|', header=None, names=itemCols)
user = pd.read_csv(io.BytesIO(uploaded['u.user']), '|', header=None, names=userCols)

# Practicum 1

## Code

In [5]:
# Normalize the ratings for user 200
user200 = data.loc[data['User'] == 200]
user200['Rating'] = user200['Rating'].apply(lambda x: x - user200['Rating'].mean())

# Normalize the ratings for user 15
user15 = data.loc[data['User'] == 15]
user15['Rating'] = user15['Rating'].apply(lambda x: x - user15['Rating'].mean())

# Get vector for movie 95
movie95 = item.loc[item['movie id'] == 95][genres]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# Get the movies that user 200 has reviewed
user200movies = item.loc[item['movie id'].isin(list(user200['Item']))][genres]
# Get the 'ideal' movie for user 200
ideal200 = []
for c in user200movies:
  ideal200.append(user200movies[c].values.dot(user200['Rating'].values)/len(user200))


# Get the movies that user 15 has reviewed
user15movies = item.loc[item['movie id'].isin(list(user15['Item']))][genres]
# Get the 'ideal' movie for user 15
ideal15 = []
for c in user15movies:
  ideal15.append(user15movies[c].values.dot(user15['Rating'].values)/len(user15))

In [7]:
print('Cosine between User 200 and Movie 95', cosine(ideal200, movie95))
print('Cosine between User 15 and Movie 95', cosine(ideal15, movie95))

Cosine between User 200 and Movie 95 0.9585827505489383
Cosine between User 15 and Movie 95 0.4203598292633993


## Write-Up

According to this recommender system, Movie 95 is much more likely to be recommended to User 200 than to User 15 since the cosine similarity between User 200's preferences and Movie 95 is **0.959**, while the cosine similarity between User 15's preferences and Movie 95 is only **0.420**.

# Practicum 2

## Code

In [0]:
# Build a dictionary from the dataframe of ratings
u = {}
for i in data.iterrows():
  _u = i[1]['User']
  _i = i[1]['Item']
  _r = i[1]['Rating']
  u.setdefault(_i, [0]*943)
  u[_i][_u - 1] = _r

In [0]:
# Translate the dictionary into a dataframe
utility = pd.DataFrame(u, columns=range(1,1683))

In [0]:
# Center the user's ratings
for i in range(943):
  utility.iloc[i] = utility.iloc[i].apply(lambda x: x - utility.iloc[i].mean())

In [11]:
# Compute the cosine similarity between user 1 and all others
similarto1 = []
for i in range(len(utility)):
  similarto1.append((i, 1 - cosine(utility.iloc[i], utility.iloc[0])))

# Sort ascending and take the last 11 so to exclude itself
sorted(similarto1, key=lambda x: x[1])[-11:]

[(520, 0.4414895704371504),
 (434, 0.4448146761910703),
 (513, 0.4453589932403397),
 (91, 0.44542969870209637),
 (822, 0.45480511185277517),
 (267, 0.4571049692646101),
 (456, 0.4571113121981709),
 (863, 0.46528274857185425),
 (737, 0.47110484000734365),
 (915, 0.48794913303276033),
 (0, 1.0)]

In [12]:
top10 = [520,434,513,91,822,267,456,863,737,915]
item508 = utility[[508]]
avg = 0
for i in top10:
  avg += item508.iloc[i]

print(avg/10)

508   -0.0522
Name: 520, dtype: float64


## Write-Up

The average centered rating for the top 10 most similar users to user 1 is **-0.0522**. 